In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [24]:
df = pd.read_csv("data/time_series.csv", index_col="Date")
df.index = pd.to_datetime(df.index)
df = df.round(4)

In [25]:
train = df.iloc[:350]
test = df.iloc[350:]

In [26]:
X_train, y_train = train.drop(["EXCESS_RETURN_T+1"], axis=1), train["EXCESS_RETURN_T+1"]
X_test, y_test = test.drop(["EXCESS_RETURN_T+1"], axis=1), test["EXCESS_RETURN_T+1"]

In [27]:
X_train.values[0]

array([ 0.3004, -0.3114,  0.5532, -0.2629,  0.3105,  1.    ,  0.7805,
       -0.1028,  0.1831,  0.8724,  0.2078,  0.3266,  0.1587, -0.63  ,
       -1.    ,  0.6786,  0.8724,  1.    , -0.84  ,  0.9583,  0.6276,
        0.16  ,  0.4959, -0.266 ,  0.9988,  0.2497,  0.0136, -0.6893,
       -0.5597, -0.0327, -0.4365,  0.8356,  0.4493,  0.4812,  0.7271,
        1.    , -1.    , -0.36  , -0.0308, -1.    , -1.    ,  0.1892,
       -0.651 , -0.6897, -0.8622, -0.547 ,  1.    , -0.4867,  0.8667])

In [28]:
y_train.values[0]

0.0385